In [3]:
import sys, os
from typing import Optional
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FRE-GT-9743-Assignments
Fixed Income Library is loaded.


## Instructions:

- The interface for Interpolator is in utilties/numerics, namely, class Interpolator1D
- The constructor asks for: absicass, ordinates, interp method (enum), and extrap method (enum)
- Make a derived "class Interpolator1DPCP" that implements the abstract methods, e.g., interpolate, integrate
- Test the code with qfCreate1DInterpolator and qfInterpolate1D

### Test Interpolation

In [ ]:
axis1 = [1, 3, 5, 7]
values = [3, 4, 5, 6]
interp_method = 'PIECEWISE_CONSTANT_LEFT_CONTINUOUS'
extrap_method = 'FLAT'
interp_1d = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
# interpolate
x = 1
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 4.}')
x = 1.5
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 4.}')
x = 3
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 5.}')
x = 5.5
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 6}')
# extrpolate left
x = 0.5
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 3}')
# extrpolate right
x = 6.5
v = qfInterpolate1D(x, interp_1d)
print(f'At {x}, the interpolated value is {v}, the diff is {v - 6}')

### Test Integration of Interpolation

In [ ]:
# 1) both out of left wing
x_s, x_e = 0.5, 0.9
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-1.2}.')
# 2) one left wing, one in the first bucket
x_s, x_e = 0.5, 1.2
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-2.3}.')
# 3) one left wing, one in the middle bucket
x_s, x_e = 0.5, 3.2
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-10.5}.')
# 4) both in the middle
x_s, x_e = 1.5, 5.2
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-17.2}.')
# 5) one in middle bucket, one on the right wing
x_s, x_e = 3.5, 7.2
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-20.7}.')
# 6) one in the last bucket, one on the right wing
x_s, x_e = 6, 7.2
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-7.2}.')
# 7) both right wing
x_s, x_e = 8, 10
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-12.}.')
# 8) complete accross
x_s, x_e = 0.1, 10
v = qfInterpolate1DIntegral(x_s, x_e, interp_1d)
print(f'From {x_s} to {x_e}, the integrand value is {v}, benchmark is {v-50.7}.')

## Instructions

- Implement sensitivity functions to compute analytic risk of interpolated value/intergarated interpolation w.r.t ordinates
    - 'gradient_wrt_ordinate'
    - 'gradient_of_integrated_value_wrt_ordinate'

- 'bump_reval_interpolator' is an bump & reval function to return the gradient of the interpolation, mimic the structure to fill out 'bump_reval_interpolator_integrand' so that it supports the gradient of intergration

- Contrast your analytic result with the bump reval result

In [ ]:
def bump_reval_interpolator(
    x : float,
    axis1 : List, 
    values : List, 
    interp_method : str,
    extrap_method : str, 
    bump_size : Optional[float]=1e-4):

    base_interpolator = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
    b_value = qfInterpolate1D(x, base_interpolator)

    grad = []
    for i in range(len(values)):
        values[i] += bump_size
        this_interp = qfCreate1DInterpolator(axis1, values, interp_method, extrap_method)
        bumped_value = qfInterpolate1D(x, this_interp)
        grad.append((bumped_value - b_value) / bump_size)
        values[i] -= bump_size

    return np.array(grad)

def bump_reval_interpolator_integrand(
    x_s : float,
    x_e : float,
    axis1 : List, 
    values : List, 
    interp_method : str,
    extrap_method : str, 
    bump_size : Optional[float]=1e-4):
    
    ### TODO: return the actual gradient

    return np.zeros(len(values))

### Interpolator Sensitivity

In [ ]:

for x in [1, 1.5, 3, 5.5, 0.5, 6.5]:
    # analytic
    grad_analytic = qfInterpolate1DGrad(x, interp_1d)
    # bump reval
    grad_br = bump_reval_interpolator(x, axis1, values, interp_method, extrap_method)
    # assertion
    print(f'With {x}, the diff is {(grad_analytic - grad_br).sum()}.')

In [ ]:
### interpolator integral sensitivity
test_cases = [
    [0.5, 0.9],
    [0.5, 1.2],
    [0.5, 3.2],
    [1.5, 5.2],
    [3.5, 7.2],
    [6, 7.2],
    [8, 10],
    [0.1, 10]
]
for x_s, x_e in test_cases:
    grad_analytic = qfInterpolate1DIntegralGrad(x_s, x_e, interp_1d)
    grad_br = bump_reval_interpolator_integrand(x_s, x_e, axis1, values, interp_method, extrap_method)
    # assertion
    print(f'With {x_s} and {x_e}, the diff is {(grad_analytic - grad_br).sum()}.')